In [93]:
import pandas as pd
import jsonlines

In [94]:
with jsonlines.open('C:/Users/krilo/PycharmProjects/ProphetNet/GENIE/GENIE/datasets/ToTTo/valid.jsonl') as f:
    df = pd.DataFrame([line for line in f.iter()])

In [95]:
MAXLENI=512
MAXLENO=128

In [96]:
from transformers import T5TokenizerFast
from transformers import T5ForConditionalGeneration
from transformers import AdamW, WarmUp, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import os
import time
import copy
import numpy
import matplotlib.pyplot as plt

In [97]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [98]:
tokenizer = T5TokenizerFast.from_pretrained("t5-base")
# model=torch.load("C:/Users/krilo/PycharmProjects/ProphetNet/GENIE/GENIE/datasets/ToTTo/T5Epoch%3A21", map_location=device)
# model.encoder.resize_token_embeddings(32128)
# model.decoder.resize_token_embeddings(32128)
state_dict = torch.load("C:/Users/krilo/PycharmProjects/Lattice/totto_data/outputs/checkpoint-150000/pytorch_model.bin")
model.load_state_dict(state_dict)

C:\Users\krilo\anaconda3\envs\DiffuSeq_env\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


<All keys matched successfully>

In [99]:
# special_tokens_dict = {'pad_token': '<pad>', 'bos_token': '<bos>', 'eos_token': '<eos>', 
#                        'additional_special_tokens': ['<PAGESTART>', '<PAGEEND>', '<SECTIONSTART>', '<SECTIONEND>',
#                                                      '<TABLESTART>','<TABLEEND>','<CELLSTART>','<CELLEND>','<COLHEADERSTART>',
#                                                      '<COLHEADEREND>','<ROWHEADERSTART>','<ROWHEADEREND>']}

# num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

# print('We have added', num_added_toks, 'tokens')
# model.encoder.resize_token_embeddings(len(tokenizer))
# model.decoder.resize_token_embeddings(len(tokenizer))

In [100]:
class tottodataset(Dataset):
  def __init__(self,df,tokenizer):
    self.sentence=df['trg']
    self.table=df['src']
    self.tokenizer=tokenizer

  def __len__(self):
    return len(self.sentence)
  
  def __getitem__(self,idx):
    inp=(self.table[idx])
#     .replace("<page_title>", "<PAGESTART>").replace("</page_title>", "<PAGEEND>") \
#                                     .replace("<section_title>", "<SECTIONSTART>").replace("</section_title>", "<SECTIONEND>") \
#                                     .replace("<table>", "<TABLESTART>").replace("</table>", "<TABLEEND>") \
#                                     .replace("<cell>", "<CELLSTART>").replace("</cell>", "<CELLEND>") \
#                                     .replace("<col_header>", "<COLHEADERSTART>").replace("</col_header>", "<COLHEADEREND>") \
#                                     .replace("<row_header>", "<ROWHEADERSTART>").replace("</row_header>", "<ROWHEADEREND>")
    out=self.sentence[idx]
    inp_tokens=self.tokenizer.encode_plus(inp, padding="max_length", max_length=MAXLENI, truncation=True)
    out_tokens=self.tokenizer.encode_plus(out, padding="max_length", max_length=MAXLENO, truncation=True)
    inp_id=inp_tokens.input_ids
    out_id=out_tokens.input_ids
    inp_mask=inp_tokens.attention_mask
    out_mask=out_tokens.attention_mask
    labels=out_tokens.input_ids.copy()
    labels=[-100  if x==self.tokenizer.pad_token_id else x for x in labels]

    return {
        "table_text":inp,
        "sentence":out,
        "input_ids":torch.tensor(inp_id, dtype=torch.long),
        "input_attention_mask":torch.tensor(inp_mask, dtype=torch.long),
        "decoder_input_ids":torch.tensor(out_id, dtype=torch.long),
        "decoder_attention_mask":torch.tensor(out_mask, dtype=torch.long),
        "labels":torch.tensor(labels, dtype=torch.long)
    }

In [101]:
test_dataset=tottodataset(df,tokenizer)

test_dataloader=DataLoader(test_dataset,
                            batch_size=8,
                            num_workers=0,
                            shuffle=False)

In [102]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [103]:
!nvidia-smi

Sat Mar  2 16:52:16 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.65       Driver Version: 511.65       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   35C    P8    11W /  N/A |   2732MiB /  6144MiB |     33%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [104]:
test_iterator=iter(test_dataloader)

In [105]:
test_batch=next(test_iterator)

In [106]:
generation_output = model.generate(test_batch['input_ids'].to(device), max_length=128, return_dict_in_generate=True, output_scores=True)

In [107]:
expected=test_batch["sentence"][0]
predicted=tokenizer.decode(generation_output["sequences"][0], skip_special_tokens=True)

In [108]:
print(f"Expected Sentence: {expected}")
print(f"Predicted Sentence: {predicted}")

Expected Sentence: Daniel Henry Chamberlain was the 76th Governor of South Carolina from 1874.
Predicted Sentence: Daniel Henry Chamberlain was the 76th Governor of South Carolina from 1874.


In [109]:
!nvidia-smi

Sat Mar  2 16:52:17 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.65       Driver Version: 511.65       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   37C    P0    30W /  N/A |   2796MiB /  6144MiB |     30%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [110]:
# bleurt_dataloader=DataLoader(test_dataset,
#                             batch_size=1,
#                             num_workers=2,
#                             shuffle=False)

In [111]:
from tqdm.notebook import tqdm

fout = open('./outputs_new_lattice.txt', 'w+', encoding='utf8')

output=[]
corrects=[]

for batch in tqdm(test_dataloader):
    input_ids=batch["input_ids"].to(device)
    
    outputs = model.generate(input_ids, return_dict_in_generate=True, output_scores=True, max_length=128)
    for i in range(8):
        try:
            outputstring = tokenizer.decode(outputs["sequences"][i], skip_special_tokens=True, clean_up_tokenization_spaces=True)

            refstring=batch["sentence"][i].replace('</s>',"")
            output.append(outputstring)
            corrects.append(refstring)

            fout.write(outputstring[:-1]+'.\n')
        except:
            pass

fout.close()

  0%|          | 0/963 [00:00<?, ?it/s]